# Let's build GTP: from scratch, in code, spelled out. 

In [19]:
with open('secret.txt', 'r', encoding="utf8") as file: 
    text = file.read()

In [26]:
print(f'length of file: {len(text)}')

length of file: 13667


In [21]:
# lets look at the first 1000 characters
print(text[:1000])

Bueno, vamos a tratar dos asuntos:

Hoy, miércoles, corresponde el ‘Quién es quién en las mentiras de la semana’, que cada vez son más las mentiras, llueven mentiras, hay tormenta de mentiras, porque hay medios que son fábricas de mentiras y de manipulación, medios de información, prensa escrita, radio, televisión, conductores de radio, conductores de televisión y también robots manejados, operados, aceitados con mucho dinero en las redes sociales, en las benditas redes sociales.

Pero, en fin, así están las cosas y hay que estar por eso informando, esa es la mejor defensa, el estar aclarando las mentiras de nuestros adversarios. Por lo general es el bloque conservador, con su grupo de élite, este poder conservador que tiene voceros, tiene intelectuales. Ya me cuesta trabajo decir ‘intelectuales’, porque quien se entrega por entero a la mentira pierde la imaginación y el talento, que eso es lo que ha venido pasando.

Además, en el periodo neoliberal, como parte de la decadencia, no sur

We are building a character-level language model, so we're just translating individual characters to integers. 

In [22]:
# here are the unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 ,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVXYZabcdefghijklmnopqrstuvwxyzº¿Ááéíñóú―‘’…
82


In [23]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string then generate a list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of ints and generate the strings 

print(encode("hii there"))
print(decode(encode("hii there")))

[50, 51, 51, 1, 62, 50, 47, 60, 47]
hii there


From what I undderstand, a ``torch.Tensor`` is a multi-dimentional matrix that contains elements of something. (Similar to a Numpy Array)

In [24]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([13667]) torch.int64
tensor([19, 63, 47, 56, 57,  2,  1, 64, 43, 55, 57, 61,  1, 43,  1, 62, 60, 43,
        62, 43, 60,  1, 46, 57, 61,  1, 43, 61, 63, 56, 62, 57, 61, 15,  0,  0,
        25, 57, 67,  2,  1, 55, 51, 73, 60, 45, 57, 54, 47, 61,  2,  1, 45, 57,
        60, 60, 47, 61, 58, 57, 56, 46, 47,  1, 47, 54,  1, 79, 34, 63, 51, 73,
        56,  1, 47, 61,  1, 59, 63, 51, 73, 56,  1, 47, 56,  1, 54, 43, 61,  1,
        55, 47, 56, 62, 51, 60, 43, 61,  1, 46, 47,  1, 54, 43,  1, 61, 47, 55,
        43, 56, 43, 80,  2,  1, 59, 63, 47,  1, 45, 43, 46, 43,  1, 64, 47, 68,
         1, 61, 57, 56,  1, 55, 72, 61,  1, 54, 43, 61,  1, 55, 47, 56, 62, 51,
        60, 43, 61,  2,  1, 54, 54, 63, 47, 64, 47, 56,  1, 55, 47, 56, 62, 51,
        60, 43, 61,  2,  1, 50, 43, 67,  1, 62, 57, 60, 55, 47, 56, 62, 43,  1,
        46, 47,  1, 55, 47, 56, 62, 51, 60, 43, 61,  2,  1, 58, 57, 60, 59, 63,
        47,  1, 50, 43, 67,  1, 55, 47, 46, 51, 57, 61,  1, 59, 63, 47,  1, 61,
        

# Final Code

In [25]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('secret.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.211922 M parameters
step 0: train loss 4.5825, val loss 4.5901
step 100: train loss 2.5221, val loss 2.5201
step 200: train loss 2.3595, val loss 2.3714
step 300: train loss 2.2756, val loss 2.3051
step 400: train loss 2.1819, val loss 2.2397
step 500: train loss 2.0679, val loss 2.1802
step 600: train loss 1.9611, val loss 2.1318
step 700: train loss 1.8601, val loss 2.1054
step 800: train loss 1.7590, val loss 2.0595
step 900: train loss 1.6655, val loss 2.0478
step 1000: train loss 1.5655, val loss 2.0714
step 1100: train loss 1.4813, val loss 2.0665
step 1200: train loss 1.3826, val loss 2.0486
step 1300: train loss 1.2992, val loss 2.0752
step 1400: train loss 1.2183, val loss 2.1188
step 1500: train loss 1.1370, val loss 2.1433
step 1600: train loss 1.0613, val loss 2.2130
step 1700: train loss 0.9839, val loss 2.2094
step 1800: train loss 0.9146, val loss 2.2870
step 1900: train loss 0.8492, val loss 2.3101
step 2000: train loss 0.8153, val loss 2.3912
step 2100: train loss 0.